# Monitor Classification Models in Verta Automatically

Verta can automatically monitor any model deployed via the Verta deployment system. 

This notebook shows how a classification model on tabular data can be monitored.

# 0. Setup

### 0.1 Imports

In [ ]:
from __future__ import print_function

import itertools
import json
import os
import time
import multiprocessing.dummy

try:
    import cloudpickle
    import numpy as np
    import pandas as pd
    import sklearn
    from sklearn import linear_model
    import verta
    from verta import Client, environment
    from verta.dataset import Path
    from verta.dataset.entities import Dataset
    from verta.environment import Python
    from verta.registry import VertaModelBase, verify_io
    from verta.registry.entities import RegisteredModel, RegisteredModelVersion
    from verta.tracking.entities import ExperimentRun
    from verta.utils import ModelAPI
    import wget
except ImportError:
    !pip install numpy pandas sklearn verta wget cloudpickle
    import cloudpickle
    import numpy as np
    import pandas as pd
    import sklearn
    from sklearn import linear_model
    import verta
    from verta import Client, environment
    from verta.dataset import Path
    from verta.dataset.entities import Dataset
    from verta.environment import Python
    from verta.registry import VertaModelBase, verify_io
    from verta.registry.entities import RegisteredModel, RegisteredModelVersion
    from verta.tracking.entities import ExperimentRun
    from verta.utils import ModelAPI
    import wget


### 0.2 Verta Client Setup

In [ ]:
# Use local env vars or uncomment and fill out the lines below:
# os.environ['VERTA_EMAIL'] = ''
# os.environ['VERTA_DEV_KEY'] = ''
# os.environ['VERTA_HOST'] = ''

client: Client = Client()

# Naming convention to be used for this example
NAME: str = 'census-clf-with-monitoring-example-eric-2'

# 1. Model Training

### 1.1 Download Example Data


In [ ]:
os.makedirs(
    os.path.dirname('data/examples/'),
    exist_ok=True,
)

# Fetch example data from our public S3 bucket.
train_data_url = "http://s3.amazonaws.com/verta-starter/census-train.csv"
train_data_filename = wget.detect_filename(train_data_url)
if not os.path.isfile(train_data_filename):
    wget.download(
        train_data_url,
        out='./data/examples/',
        )

df_train: pd.DataFrame = pd.read_csv(f"./data/examples/{train_data_filename}")

test_data_url = "http://s3.amazonaws.com/verta-starter/census-test.csv"
test_data_filename = wget.detect_filename(test_data_url)
if not os.path.isfile(test_data_filename):
    wget.download(
        test_data_url,
        out='./data/examples/',
        )

df_test: pd.DataFrame = pd.read_csv(f"./data/examples/{test_data_filename}")

In [ ]:
dtypes = dict()
# Note: accurate data types are required for monitoring to use the correct metrics

for column in df_train.columns:
    if column in ["age","capital-gain","capital-loss","hours-per-week"]:
        dtypes[column] = int
    else:
        dtypes[column] = bool # turn int to bool to capture binary nature of the remaining columns

df_train = df_train.astype(dtypes)

X_train = df_train.iloc[:, :-1]
Y_train = df_train.iloc[:, -1]


### 1.2 Fit Model

In [ ]:
model = linear_model.LogisticRegression(C=1e-5, solver='lbfgs', max_iter=100)
model.fit(X_train, Y_train)

# 2.0 Create and Register a Model


### 2.1 Define Model
Create a very basic example model with the minimum required functions (`init` and `predict`)

In [ ]:
class CensusIncomeClassifier(VertaModelBase):
    def __init__(self, artifacts):
        self.model = cloudpickle.load(open(artifacts["serialized_model"], "rb"))

    @verify_io
    def predict(self, batch_input):
        # Model produces True/False that is being turned into 0/1
        prediction = map(lambda p : 1 if p else 0, self.model.predict(batch_input).tolist())
        confidence_percentage = self.model.predict_proba(batch_input).max(axis=1).tolist()
        return list(zip(prediction, confidence_percentage))

In [ ]:
with open("./data/model.pkl", "wb") as f:
    cloudpickle.dump(model, f)
artifacts_dict = {"serialized_model" : "./data/model.pkl"}

### 2.2 Register the Model in Verta

In [ ]:
registered_model: RegisteredModel = client.get_or_create_registered_model(name=NAME)

# Note: As of Verta Release 2022.04, confidence scores are recommended for classification models
# in order to accurate compute ROC and PR curves.

# Code below adds a dummy confidence column to the output to register this column in the system
# Note the naming convention of ".confidence" to identify the column as the confidence score
Y_train_with_confidence: pd.DataFrame = pd.merge(
    Y_train.to_frame(),
    0. * Y_train.to_frame().rename(columns={Y_train.name: Y_train.name + ".confidence"}),
    left_index=True,
    right_index=True)

model_version: RegisteredModelVersion = registered_model.create_standard_model(
    name = "v1",
    model_cls = CensusIncomeClassifier,
    model_api = ModelAPI(X_train, Y_train_with_confidence),
    environment = Python(requirements=["scikit-learn"]),
    artifacts = artifacts_dict
)

### 2.3 Log Reference Data

Upload reference data as a dataset version. This is your training dataset and will help facilitate downstream drift monitoring against this reference set. You do not need to upload your entire training set, but a statistically significant representation that mirrors your training/reference data distribution.

In [ ]:
# Fetch downloaded training data
dataset: Dataset = client.get_or_create_dataset(NAME)
content: Path = Path([f"./data/examples/{train_data_filename}"], enable_mdb_versioning=True)
dataset_version = dataset.create_version(content)

In [ ]:
model_version.log_dataset_version(key='reference', dataset_version=dataset_version)

## 3. Deploy Model and Start Monitoring

Once a model is deployed, along with a live endpoint, a new entry is created in the model monitoring tab with the endpoint name, default monitoring dashboards are created and all the features, predictions and drift alerts are configured. 

In [ ]:
endpoint = client.get_or_create_endpoint(NAME)
endpoint.update(model_version, wait=True)

### 3.1 Make Predictions and Log Ground-truth

The next snippets of code simulate the real-world where:

- caller sends input data for prediction
- model makes a prediction and assigs a unique UUID for the prediction
- ground truth is then registered with the system using the above UUID

Once the data has been sent to the system, you can navigate to the webapp to view dashboards

In [ ]:
def simulate_predictions(endpoint, deployed_model, input_data, ground_truth, col_name, ground_truth_delay): 
    # ground_truth_delay is delay in seconds between prediction & GT becoming available
    import time
    
    ids = []
    for i, row in input_data.iterrows():
        _id, _ = deployed_model.predict_with_id([row.tolist()])
        ids.append(_id)

    time.sleep(ground_truth_delay)
    id_and_gt = zip(ids, ground_truth)
    for t in id_and_gt:
        endpoint.log_ground_truth(t[0], [t[1]], col_name) # id, gt, prediction_col_name

In [ ]:
deployed_model = endpoint.get_deployed_model()

In [ ]:
simulate_predictions(endpoint, deployed_model, df_test.iloc[:100, :-1], df_test.iloc[:100, -1], ">50k", 10)

### 3.2 Introduce Drift into the Data

In [ ]:
skewed_age_input_features = df_test.copy()
skewed_age_input_features['age'] = skewed_age_input_features['age'] + 20

simulate_predictions(endpoint, deployed_model, 
                     skewed_age_input_features.iloc[600:700, :-1], 
                     df_test.iloc[600:700, -1], ">50k", 10)